In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
import pickle
# import pandas as pd
from IPython.display import Image, display

from keras.preprocessing import image, sequence
from keras.layers import Dense, Convolution2D, Dropout, LSTM,Input, TimeDistributed, Embedding, Add, Activation, RepeatVector
from keras.models import Sequential, Model
# from keras.optimizers import Adam
from keras.applications import ResNet50

ModuleNotFoundError: No module named 'keras'

# Loading Dictionaries

In [ ]:
with open('./indices_2_words.p','rb') as f1:
    indices_2_words = pickle.load(f1)

with open('./word_2_indices.p','rb') as f2:
    words_2_indices = pickle.load(f2)

f1.close()
f2.close()
vocab_size = len(indices_2_words)
print(len(indices_2_words))
print(len(words_2_indices))

In [ ]:
print(indices_2_words[456])
print(words_2_indices['backpacker'])
max_len= 40

# Predicting captions

In [ ]:
model = ResNet50(include_top=False,weights='imagenet',input_shape=(224,224,3),pooling='avg')

In [ ]:
embedding_size = 128
max_len = 40

In [ ]:
image_model_inp = Input(shape=(2048,))
image_model_ly1 = Dense(embedding_size, activation='relu')(image_model_inp)
image_model_ly2 = RepeatVector(max_len)(image_model_ly1)

image_model = Model(input =image_model_inp,output = image_model_ly2)
image_model.summary()

In [ ]:
language_model_inp = Input(shape=(max_len,))
language_model_ly1 = Embedding(input_dim=vocab_size, output_dim=embedding_size)(language_model_inp)
language_model_ly2 = LSTM(256, return_sequences=True)(language_model_ly1)
language_model_ly3 = TimeDistributed(Dense(embedding_size))(language_model_ly2)

language_model = Model(input=language_model_inp,output=language_model_ly3)
language_model.summary()

In [ ]:
model_ly1 = Add()([image_model_ly2, language_model_ly3])
model_ly2 = LSTM(128, return_sequences=True)(model_ly1)
model_ly3 = LSTM(512, return_sequences=False)(model_ly2)
model_ly4 = Dense(vocab_size,activation = 'softmax')(model_ly3)


new_model = Model(input = [image_model_inp,language_model_inp],output = model_ly4)
new_model.load_weights("./model_weights.h5")


In [ ]:
def Image_preprocessing(img_path):
    img = image.load_img(img_path,target_size=(224,224))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img_emb = model.predict(img).reshape(2048)
#     print(type(img_emb))
    return img_emb   

In [ ]:
img_path = "../../project/Flickr8k_Dataset/Flicker8k_Dataset/1386251841_5f384a0fea.jpg"
test_img = Image_preprocessing(img_path)

In [ ]:
def predict_captions(image):
    start_word = ["<start>"]
    while True:
        par_caps = [words_2_indices[i] for i in start_word]
        par_caps = sequence.pad_sequences([par_caps], maxlen=max_len, padding='post')
        preds = new_model.predict([np.array([image]), np.array(par_caps)])
        word_pred = indices_2_words[np.argmax(preds[0])]
        start_word.append(word_pred)
        
        if word_pred == "<end>" or len(start_word) > max_len:
            break
            
    return ' '.join(start_word[1:-1])


In [ ]:
z = Image(filename=img_path)
display(z)

print (predict_captions(test_img))